In [1]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv1D, Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
from itertools import groupby
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
from sklearn.preprocessing import StandardScaler
from itertools import combinations
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
revealed_targets = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')

In [3]:
train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [4]:
import pandas as pd

def apply_differencing(df, columns_to_difference):
    """
    Apply pandas diff() method to specified columns in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to process.
    columns_to_difference (list of str): List of column names to apply differencing.

    Returns:
    pandas.DataFrame: DataFrame with differenced columns.
    """
    for column in columns_to_difference:
        if column in df.columns:
            df[f'{column}_diff'] = df[column].diff()
        else:
            print(f"Column {column} not found in DataFrame.")
    return df


In [5]:
def create_lagged_features(df, columns, n_lags, fill_method='ffill'):

    lagged_df = df.copy()
    for col in columns:
        # lagged features for each column
        for n in range(1, n_lags + 1):
            lagged_col_name = f'{col}_lag{n}'
            lagged_df[lagged_col_name] = df[col].shift(n)
            # fill method after each lagged column
            if fill_method == 'ffill':
                lagged_df[lagged_col_name].fillna(method='ffill', inplace=True)
            elif fill_method == 'zero':
                lagged_df[lagged_col_name].fillna(0, inplace=True)

    return lagged_df

In [6]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def impute_missing_values(df, columns_to_impute, max_iter=10, random_state=0):
    """
    Impute missing values in specified columns of a DataFrame using IterativeImputer.

    Parameters:
    df (pandas.DataFrame): The DataFrame to process.
    columns_to_impute (list of str): List of column names to apply imputation.
    max_iter (int): Maximum number of imputation iterations.
    random_state (int): Random state for reproducibility.

    Returns:
    pandas.DataFrame: DataFrame with imputed columns.
    """
    # Check if columns_to_impute are in df
    columns_to_impute = [col for col in columns_to_impute if col in df.columns]

    # Select the columns for imputation
    columns_data = df[columns_to_impute]

    # Check if there are any missing values and if the columns are numeric
    if columns_data.isna().any().any() and columns_data.select_dtypes(include='number').shape[1] == len(columns_to_impute):
        imputer = IterativeImputer(max_iter=max_iter, random_state=random_state)

        # Apply imputation
        imputed_data = imputer.fit_transform(columns_data)
        imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute, index=df.index)

        # Update the original DataFrame
        df.update(imputed_df)

    return df


In [7]:
import pandas as pd

def preprocess_and_interpolate(df, columns_to_interpolate, start_time, freq='T', order=3):
    """
    Preprocess and interpolate specified columns of a DataFrame.

    Parameters:
    df (pandas.DataFrame): DataFrame to process.
    columns_to_interpolate (list of str): Columns to apply interpolation.
    start_time (str or datetime): Start time for generating the time range.
    freq (str): Frequency for resampling. Default is 'T' for minute-wise.
    order (int): Order of spline interpolation. Default is 3.

    Returns:
    pandas.DataFrame: Processed and interpolated DataFrame.
    """
    # Generate a time range and map 'date_id' to the time_range
    time_range = pd.date_range(start=start_time, periods=len(df), freq=freq)
    df['actual_time'] = df['date_id'].apply(lambda x: time_range[x])

    # Set the timestamp as the index
    df.set_index('actual_time', inplace=True)

    # Initialize an empty DataFrame to store the results
    resampled_dfs = []

    # Group by 'stock_id' and process each group
    for stock_id, group in df.groupby('stock_id'):
        # Handle duplicates
        if group.index.duplicated().any():
            group = group[~group.index.duplicated(keep='first')]

        # Resample the data
        group_resampled = group.resample(freq).asfreq()

        # Interpolate for each column
        for column in columns_to_interpolate:
            group_resampled[column] = group_resampled[column].interpolate(method='spline', order=order)
        
        # Add the stock_id back to the DataFrame
        group_resampled['stock_id'] = stock_id

        # Append the processed group to the list
        resampled_dfs.append(group_resampled)

    # Concatenate all the processed groups
    df_resampled = pd.concat(resampled_dfs)
    df_resampled.reset_index(inplace=True)

    return df_resampled




In [8]:
import pandas as pd

def preprocess_data(df, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, fill_method='ffill', start_time=None, freq='T', order=3):
    """
    Apply a series of preprocessing steps: differencing, imputing, interpolating, and creating lagged features.

    Parameters:
    df (pandas.DataFrame): The DataFrame to process.
    columns_to_difference (list of str): Columns to apply differencing.
    columns_to_impute (list of str): Columns to apply imputation.
    columns_to_interpolate (list of str): Columns to apply interpolation.
    lagged_columns (list of str): Columns to create lagged features for.
    n_lags (int): Number of lags to create.
    fill_method (str): Method for filling NaN values in lagged features.
    start_time (str or datetime): Start time for time range generation in interpolation.
    freq (str): Frequency for resampling in interpolation.
    order (int): Order of spline interpolation.

    Returns:
    pandas.DataFrame: The processed DataFrame.
    """
    # Apply differencing
    df = apply_differencing(df, columns_to_difference)
    df.dropna(subset=columns_to_difference)

    # Impute missing values
    df = impute_missing_values(df, columns_to_impute)

    # Interpolate
    if start_time:
        df = preprocess_and_interpolate(df, columns_to_interpolate, start_time, freq, order)

    # Create lagged features
    df = create_lagged_features(df, lagged_columns, n_lags, fill_method)

    return df

# processed_df = preprocess_data(train, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, fill_method='ffill', start_time=start_time, freq='T', order=3)


In [9]:
columns_to_difference = ['bid_price', 'ask_price']
columns_to_impute = ['far_price', 'near_price']
columns_to_interpolate = ['bid_price', 'ask_price']
lagged_columns = ['far_price', 'near_price']
n_lags = 5
start_time = pd.to_datetime('2023-01-01 08:00:00')


In [10]:
def preprocess_train_data(df, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, fill_method, start_time, freq, order):
    """
    Apply a full preprocessing pipeline to the DataFrame.

    Parameters:
    df (pandas.DataFrame): DataFrame to process.
    columns_to_difference (list of str): Columns to apply differencing.
    columns_to_impute (list of str): Columns to apply imputation.
    columns_to_interpolate (list of str): Columns to apply interpolation.
    lagged_columns (list of str): Columns to create lagged features for.
    n_lags (int): Number of lags to create.
    fill_method (str): Method for filling NaN values in lagged features.
    start_time (datetime): Start time for generating the time range.
    freq (str): Frequency for resampling in interpolation.
    order (int): Order of spline interpolation.

    Returns:
    numpy.ndarray: Scaled feature array.
    numpy.ndarray: Target array (if available).
    """
    # Apply differencing, imputation, interpolation, and create lagged features
    df = preprocess_data(df, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, fill_method, start_time, freq, order)

    # Replace inf values with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    # Check if 'target' column exists and separate it
    if 'target' in df.columns:
        y = df['target'].values
        df = df.drop(['target'], axis=1)
    else:
        y = None

    # Drop 'actual_time' column if it exists
    if 'actual_time' in df.columns:
        df = df.drop(['actual_time'], axis=1)

    # Convert DataFrame to numpy array and scale the features
    X = df.to_numpy(dtype='float32')
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, y

# Example usage
processed_X, processed_y = preprocess_train_data(train, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, 'ffill', start_time, 'T', 3)


In [11]:
def preprocess_test_data(df, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, fill_method, start_time, freq, order):
    """
    Apply preprocessing steps to the test DataFrame.

    Parameters:
    df (pandas.DataFrame): DataFrame to process.
    columns_to_difference (list of str): Columns to apply differencing.
    columns_to_impute (list of str): Columns to apply imputation.
    columns_to_interpolate (list of str): Columns to apply interpolation.
    lagged_columns (list of str): Columns to create lagged features for.
    n_lags (int): Number of lags to create.
    fill_method (str): Method for filling NaN values in lagged features.
    start_time (datetime): Start time for generating the time range.
    freq (str): Frequency for resampling in interpolation.
    order (int): Order of spline interpolation.

    Returns:
    numpy.ndarray: Scaled feature array.
    """
    # Apply differencing, imputation, interpolation, and create lagged features
    df = preprocess_data(df, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, fill_method, start_time, freq, order)

    # Replace inf values with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    # Drop 'actual_time' column if it exists
    if 'actual_time' in df.columns:
        df = df.drop(['actual_time'], axis=1)

    # Convert DataFrame to numpy array and scale the features
    X = df.to_numpy(dtype='float32')
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled

# Example usage
processed_test_X = preprocess_test_data(test, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, 'ffill', start_time, 'T', 3)


In [12]:
processed_X

array([[-1.7158246, -1.707438 ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-1.7158246, -1.7002189,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-1.7158246, -1.693    ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [ 1.7229621,  1.7071792,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 1.7229621,  1.7143983,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 1.7229621,  1.7216173,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [13]:
processed_test_X

array([[-1.7349446 ,  1.2221756 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.7174786 , -1.2262908 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.7174786 , -0.00205753,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.7233202 , -1.2262908 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.7233202 , -0.00205753,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.7233202 ,  1.2221756 ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv1D, Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization


class TemporalBlock(Layer):
    def __init__(self, n_filters, kernel_size, dilation_rate):
        super(TemporalBlock, self).__init__()
        self.conv1 = Conv1D(
            filters=n_filters,
            kernel_size=kernel_size,
            padding='causal',
            dilation_rate=dilation_rate
        )
        self.batch_norm1 = BatchNormalization()
        self.conv2 = Conv1D(
            filters=n_filters,
            kernel_size=kernel_size,
            padding='causal',
            dilation_rate=dilation_rate
        )
        self.batch_norm2 = BatchNormalization()
        self.dropout = Dropout(0.2)
    
    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.batch_norm1(x, training=training)
        x = tf.nn.relu(x)
        x = self.dropout(x, training=training)
        x = self.conv2(x)
        x = self.batch_norm2(x, training=training)
        x = tf.nn.relu(x)
        x = self.dropout(x, training=training)
        return x + inputs  # Residual connection

class TCN(Layer):
    def __init__(self, n_filters, kernel_size, dilation_rates):
        super(TCN, self).__init__()
        self.blocks = [TemporalBlock(n_filters, kernel_size, dr) for dr in dilation_rates]

    def call(self, inputs, training=False):
        x = inputs
        for block in self.blocks:
            x = block(x, training=training)
        return x

# Define the TCN model
n_filters = 64
kernel_size = 3
dilation_rates = [1, 2, 4, 8]  # exadilation rates

input_layer = Input(shape=(None, 1))  # univariate time series; change 1 to n_features if multivariate
tcn_layer = TCN(n_filters, kernel_size, dilation_rates)(input_layer)
output_layer = Dense(1)(tcn_layer)  # regression task

m = Model(inputs=input_layer, outputs=output_layer)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, clipvalue=1.0)
m.compile(optimizer=optimizer, loss='mae')
m.summary()
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
m.fit(processed_X, processed_y, epochs=1, callbacks=[early_stopping])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 tcn (TCN)                   (None, None, 64)          88768     
                                                                 
 dense (Dense)               (None, None, 1)           65        
                                                                 
Total params: 88,833
Trainable params: 87,809
Non-trainable params: 1,024
_________________________________________________________________
2976/2976 [==============================] - 121s 39ms/step - loss: 8.0163


In [15]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
counter = 0
for (test_batch, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test_batch.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))
    X_feat = preprocess_test_data(test_batch, columns_to_difference, columns_to_impute, columns_to_interpolate, lagged_columns, n_lags, 'ffill', start_time, 'T', 3)
    sample_prediction['target'] = m.predict(X_feat)
    env.predict(sample_prediction)
    counter += 1
